In [ ]:
# mit sortierfunktion 
import tkinter as tk
from tkinter import ttk
import pvlib
import numpy as np

# Daten aus der Sandia-Moduldatenbank laden
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

# Sortierfunktion
class SortableTreeview(ttk.Treeview):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self._sort_column = None
        self._sort_reverse = False

    def heading(self, column, text=None, **kwargs):
        if text:
            super().heading(column, text=text, command=lambda _col=column: self.sort_column(_col))
        else:
            super().heading(column, **kwargs)

    def sort_column(self, col):
        l = [(self.set(k, col), k) for k in self.get_children('')]
        l.sort(reverse=self._sort_reverse)
        
        for index, (val, k) in enumerate(l):
            self.move(k, '', index)
        
        self._sort_reverse = not self._sort_reverse
        self._sort_column = col

# GUI erstellen
class ModuleDatabaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sandia Moduldatenbank")
        
        self.create_widgets()
        
        # Daten für die Anzeige
        self.displayed_modules = list(sandia_modules.keys())  # Alle Module zu Beginn anzeigen
        
        # Einfügen der Daten in die Treeview
        self.update_displayed_modules()
        
        # Doppelklick-Event für Treeview-Einträge
        self.tree.bind("<Double-1>", self.on_double_click)
        
        # Event für das Resize des Hauptfensters
        self.root.bind("<Configure>", self.on_resize)

    def create_widgets(self):
        # Suchfeld
        self.search_var = tk.StringVar()
        self.search_var.trace("w", self.update_displayed_modules)
        self.search_entry = ttk.Entry(self.root, textvariable=self.search_var)
        self.search_entry.pack(padx=10, pady=10, fill=tk.X)

        # Treeview für die Anzeige der Module
        self.tree = SortableTreeview(self.root)
        self.tree["columns"] = ("Modul", "Vintage", "Area", "Material", "Cells_in_Series",
                                "Parallel_Strings", "Isco", "Voco", "Impo", "Vmpo")
        

        # Entfernen der #0-Spalte
        self.tree.column("#0", width=0, stretch=tk.NO)
        self.tree.heading("#0", text="")

        # Festlegen der Spaltenüberschriften
        for col in self.tree["columns"]:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        
        self.tree.column("Modul", width=150)

        # Packen der Treeview
        self.tree.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        # Schließen-Button
        self.close_button = ttk.Button(self.root, text="Schließen", command=self.on_close)
        self.close_button.pack(pady=10)

    def update_displayed_modules(self, *args):
        # Funktion zum Aktualisieren der angezeigten Module basierend auf der Sucheingabe
        search_text = self.search_var.get().lower()  # Suchtext ohne Groß-/Kleinschreibung

        # Filtern der Module
        self.displayed_modules = [module for module in sandia_modules
                                  if search_text in module.lower()]

        # Löschen aller bestehenden Einträge in der Treeview
        self.tree.delete(*self.tree.get_children())

        # Einfügen der gefilterten Module in die Treeview
        for module in self.displayed_modules:
            data = sandia_modules[module]
            self.tree.insert("", "end", values=(
                module, data['Vintage'], data['Area'], data['Material'],
                data['Cells_in_Series'], data['Parallel_Strings'],
                data['Isco'], data['Voco'], data['Impo'], data['Vmpo']
            ))

    def on_double_click(self, event):
        # Funktion, die aufgerufen wird, wenn ein Eintrag in der Treeview doppelt geklickt wird
        item = self.tree.selection()[0]  # Auswahl des Eintrags
        module_name = self.tree.item(item, "values")[0]  # Name des Moduls aus dem ersten Wert der values

        # Neues Fenster für die Detailansicht des Moduls
        module_details_window = tk.Toplevel(self.root)
        module_details_window.title(f"Details für Modul: {module_name}")
        module_details_window.minsize(200, 900)

        # Treeview für die Detailansicht
        self.detail_tree = SortableTreeview(module_details_window)
        self.detail_tree["columns"] = ("Wert")
        self.detail_tree.heading("#0", text="Parameter")
        self.detail_tree.column("#0", width=150)
        self.detail_tree.heading("Wert", text="Wert")
        self.detail_tree.column("Wert", width=200)

        # Daten für das ausgewählte Modul
        module_data = sandia_modules[module_name]

        # Einfügen der Daten in den Treeview
        for key, value in module_data.items():
            self.detail_tree.insert("", "end", text=key, values=(str(value)))

        # Packen und Konfigurieren des Treeviews für die Detailansicht
        self.detail_tree.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

    def on_resize(self, event):
        # Funktion, die aufgerufen wird, wenn das Hauptfenster neu dimensioniert wird
        # Treeview im Hauptfenster anpassen
        self.tree.pack_configure(fill=tk.BOTH, expand=True)
        
        # Treeview in Detailfenstern anpassen, falls geöffnet
        if hasattr(self, 'detail_tree'):
            self.detail_tree.pack_configure(fill=tk.BOTH, expand=True)

    def on_close(self):
        self.root.quit()
        self.root.destroy()

# Hauptprogramm
if __name__ == "__main__":
    root = tk.Tk()
    app = ModuleDatabaseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)  # Sauberes Schließen
    root.mainloop()


In [5]:
# CEC Mode

import tkinter as tk
from tkinter import ttk
import pvlib

# Daten aus der Sandia-Moduldatenbank laden
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')

# Sortierfunktion für Treeview
class SortableTreeview(ttk.Treeview):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self._sort_column = None
        self._sort_reverse = False

    def heading(self, column, text=None, **kwargs):
        if text:
            super().heading(column, text=text, command=lambda _col=column: self.sort_column(_col))
        else:
            super().heading(column, **kwargs)

    def sort_column(self, col):
        data = [(self.set(child, col), child) for child in self.get_children('')]
        data.sort(reverse=self._sort_reverse)

        for index, (val, child) in enumerate(data):
            self.move(child, '', index)

        self._sort_reverse = not self._sort_reverse
        self._sort_column = col

# GUI erstellen
class ModuleDatabaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("CEC Moduldatenbank")

        self.create_widgets()

        # Daten für die Anzeige
        self.displayed_modules = list(cec_modules.keys())  # Alle Module zu Beginn anzeigen

        # Einfügen der Daten in die Treeview
        self.update_displayed_modules()

        # Doppelklick-Event für Treeview-Einträge
        self.tree.bind("<Double-1>", self.on_double_click)

        # Event für das Resize des Hauptfensters
        self.root.bind("<Configure>", self.on_resize)

    def create_widgets(self):
        # Suchfeld
        self.search_var = tk.StringVar()
        self.search_var.trace("w", self.update_displayed_modules)
        self.search_entry = ttk.Entry(self.root, textvariable=self.search_var)
        self.search_entry.pack(padx=10, pady=10, fill=tk.X)

        # Treeview für die Anzeige der Module
        self.tree = SortableTreeview(self.root)
        self.tree["columns"] = (
            "Technology", "Bifacial", "STC", "PTC", "A_c", "Length", "Width",
            "N_s", "I_sc_ref", "V_oc_ref", "I_mp_ref", "V_mp_ref", "alpha_sc",
            "beta_oc", "T_NOCT", "a_ref", "I_L_ref", "I_o_ref", "R_s", "R_sh_ref",
            "Adjust", "gamma_r", "BIPV", "Version", "Date"
        )

        # Entfernen der #0-Spalte
        self.tree.column("#0", width=0, stretch=tk.NO)
        self.tree.heading("#0", text="")

        # Festlegen der Spaltenüberschriften
        columns = (
            "Technology", "Bifacial", "STC", "PTC", "A_c", "Length", "Width",
            "N_s", "I_sc_ref", "V_oc_ref", "I_mp_ref", "V_mp_ref", "alpha_sc",
            "beta_oc", "T_NOCT", "a_ref", "I_L_ref", "I_o_ref", "R_s", "R_sh_ref",
            "Adjust", "gamma_r", "BIPV", "Version", "Date"
        )
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)

        # Packen der Treeview
        self.tree.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        # Schließen-Button
        self.close_button = ttk.Button(self.root, text="Schließen", command=self.on_close)
        self.close_button.pack(pady=10)

    def update_displayed_modules(self, *args):
        # Funktion zum Aktualisieren der angezeigten Module basierend auf der Sucheingabe
        search_text = self.search_var.get().lower()  # Suchtext ohne Groß-/Kleinschreibung

        # Filtern der Module
        self.displayed_modules = [module for module in cec_modules
                                  if search_text in module.lower()]

        # Löschen aller bestehenden Einträge in der Treeview
        self.tree.delete(*self.tree.get_children())

        # Einfügen der gefilterten Module in die Treeview
        for module in self.displayed_modules:
            data = cec_modules[module]
            self.tree.insert("", "end", values=(
                data['Technology'], data['Bifacial'], data['STC'], data['PTC'],
                data['A_c'], data['Length'], data['Width'], data['N_s'],
                data['I_sc_ref'], data['V_oc_ref'], data['I_mp_ref'], data['V_mp_ref'],
                data['alpha_sc'], data['beta_oc'], data['T_NOCT'], data['a_ref'],
                data['I_L_ref'], data['I_o_ref'], data['R_s'], data['R_sh_ref'],
                data['Adjust'], data['gamma_r'], data['BIPV'], data['Version'], data['Date']
            ))

    def on_double_click(self, event):
        # Funktion, die aufgerufen wird, wenn ein Eintrag in der Treeview doppelt geklickt wird
        item = self.tree.selection()[0]  # Auswahl des Eintrags
        module_name = self.tree.item(item, "text")  # Name des Moduls aus dem ersten Wert der values

        # Neues Fenster für die Detailansicht des Moduls
        module_details_window = tk.Toplevel(self.root)
        module_details_window.title(f"Details für Modul: {module_name}")
        module_details_window.minsize(200, 900)

        # Treeview für die Detailansicht
        detail_tree = ttk.Treeview(module_details_window)
        detail_tree["columns"] = ("Wert",)
        detail_tree.heading("#0", text="Parameter")
        detail_tree.column("#0", width=150)
        detail_tree.heading("Wert", text="Wert")
        detail_tree.column("Wert", width=200)

        # Daten für das ausgewählte Modul
        module_data = cec_modules[module_name]

        # Einfügen der Daten in den Treeview
        for key, value in module_data.items():
            detail_tree.insert("", "end", text=key, values=(str(value)))

        # Packen und Konfigurieren des Treeviews für die Detailansicht
        detail_tree.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

    def on_resize(self, event):
        # Funktion, die aufgerufen wird, wenn das Hauptfenster neu dimensioniert wird
        # Treeview im Hauptfenster anpassen
        self.tree.pack_configure(fill=tk.BOTH, expand=True)

    def on_close(self):
        self.root.quit()
        self.root.destroy()

# Hauptprogramm
if __name__ == "__main__":
    root = tk.Tk()
    app = ModuleDatabaseApp(root)
    root.protocol("WM_DELETE_WINDOW", app.on_close)  # Sauberes Schließen
    root.mainloop()


In [3]:
import pvlib
# cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')
sandia_modules = pvlib.pvsystem.retrieve_sam('sandiamod')


In [ ]:
%reset -f

# DB Export 

In [ ]:
import pandas as pd
import pvlib

# Laden der Sandia-Moduldatenbank direkt mit retrieve_sam() und dem Namen 'sandiamod'
sandia_modules = pvlib.pvsystem.retrieve_sam(name='sandiamod')
cec_modules = pvlib.pvsystem.retrieve_sam('CECMod')

# Konvertieren in einen DataFrame
sandia_df = pd.DataFrame(sandia_modules)
cec_df = pd.DataFrame(cec_modules)

# Anzeigen des DataFrames
# print(df.shape)
# print(df.head())
# print(df.columns)
# print(df.describe())

# Export

# excel_file = 'ModuleDB.xlsx'
# with pd.ExcelWriter(excel_file) as writer:
#     sandia_df.to_excel(writer, sheet_name='sandia_modules')
#     cec_df.to_excel(writer, sheet_name='cec_modules')

# print(f"Die DataFrames wurden erfolgreich in der Datei '{excel_file}' gespeichert.")
cec_df.to_csv('CECModuleDB.csv', ";")
print(sandia_df.info())
print(sandia_df)


In [ ]:
import pandas as pd

# Beispiel DataFrames
df1 = pd.DataFrame({
    'A': [1, 2, 3, 4],
    'B': [10, 20, 30, 40]
}, index=['a', 'b', 'c', 'd'])  # Beispiel für unterschiedliche Indizes

df2 = pd.DataFrame({
    'X': ['a', 'b', 'c', 'd'],
    'Y': ['apple', 'banana', 'cherry', 'date']
})

# Dateipfad für die Excel-Datei
excel_file = 'zwei_dataframes_mit_eigenen_indizes.xlsx'

# Excel-Writer erstellen
with pd.ExcelWriter(excel_file) as writer:
    # DataFrames in separate Sheets speichern
    df1.to_excel(writer, sheet_name='Sheet1')
    df2.to_excel(writer, sheet_name='Sheet2')

print(f"Die DataFrames wurden erfolgreich in der Datei '{excel_file}' gespeichert.")


# tk inter Testdarstellungen 

In [ ]:
# tk inter darstellung 
import tkinter as tk

root = tk.Tk()

# Setting the title
root.title("My Application")

# Setting the size and position
root.geometry("800x600+100+50")

# Setting the minimum and maximum size
root.minsize(400, 300)
root.maxsize(1200, 800)

# Making the window resizable
root.resizable(True, True)


# Setting the background color
root.configure(bg="light blue")

# Setting the initial state to maximized
root.state('zoomed')

# Setting window attributes
root.attributes('-alpha', 0.4)
root.attributes('-topmost', True)

# Starting the Tkinter event loop
root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk

class SortableTreeview(ttk.Treeview):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self._heading_config = {}

    def heading(self, column, text=None, **kwargs):
        if text:
            self._heading_config[column] = {'text': text, **kwargs}
            super().heading(column, text=text, command=lambda _col=column: self.sort_column(_col, False), **kwargs)
        else:
            super().heading(column, **kwargs)

    def sort_column(self, col, reverse):
        l = [(self.set(k, col), k) for k in self.get_children('')]
        l.sort(reverse=reverse)
        for index, (val, k) in enumerate(l):
            self.move(k, '', index)
        self.heading(col, command=lambda: self.sort_column(col, not reverse))

def on_treeview_column_click(tv, col):
    tv.sort_column(col, False)

root = tk.Tk()
root.title("Sortable Treeview")

tree = SortableTreeview(root, columns=("Name", "Age", "City"), show="headings")
tree.pack(fill=tk.BOTH, expand=True)

tree.heading("Name", text="Name")
tree.heading("Age", text="Age")
tree.heading("City", text="City")

tree.column("Name", width=150)
tree.column("Age", width=50)
tree.column("City", width=100)

# Add some sample data
data = [
    ("Alice", 30, "New York"),
    ("Bob", 25, "Los Angeles"),
    ("Charlie", 35, "Chicago"),
    ("David", 20, "Houston"),
]

for person in data:
    tree.insert("", "end", values=person)

root.mainloop()


In [ ]:
import tkinter as tk
from tkinter import ttk

# Füge hier die SortableTreeview-Klasse hinzu (wie in Schritt 1 definiert)
class SortableTreeview(ttk.Treeview):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self._heading_config = {}

    def heading(self, column, text=None, **kwargs):
        if text:
            self._heading_config[column] = {'text': text, **kwargs}
            super().heading(column, text=text, command=lambda _col=column: self.sort_column(_col, False), **kwargs)
        else:
            super().heading(column, **kwargs)

    def sort_column(self, col, reverse):
        l = [(self.set(k, col), k) for k in self.get_children('')]
        l.sort(reverse=reverse)
        for index, (val, k) in enumerate(l):
            self.move(k, '', index)
        self.heading(col, command=lambda: self.sort_column(col, not reverse))

# Verwendung der SortableTreeview-Klasse anstelle der ttk.Treeview-Klasse
root = tk.Tk()
root.title("My Application")

tree = SortableTreeview(root, columns=("Name", "Age", "City"), show="headings")
tree.pack(fill=tk.BOTH, expand=True)

tree.heading("Name", text="Name")
tree.heading("Age", text="Age")
tree.heading("City", text="City")

tree.column("Name", width=150)
tree.column("Age", width=50)
tree.column("City", width=100)

data = [
    ("Alice", 30, "New York"),
    ("Bob", 25, "Los Angeles"),
    ("Charlie", 35, "Chicago"),
    ("David", 20, "Houston"),
]

for person in data:
    tree.insert("", "end", values=person)

root.mainloop()
